In [ ]:
# DO NOT DELETE THIS CELL
# ADD YOUR PARAMETER DEFAULT VALUES HERE
sales_transactions = None

In [ ]:
import os
import pandas as pd

In [ ]:
from dotenv import load_dotenv
load_dotenv()
env_mode = os.environ["ENV_MODE"]
sql_connection_string = os.environ["SQL_CONNECTION_STRING"]

def is_dev_mode(env_mode: str):
    """
    check if the current env_mode is "dev", otherwise "prod" is assumed
    """
    return env_mode == "dev"

def debug(var):
    """
    only debug (print) the variables if env_mode is "dev"
    """
    if is_dev_mode(env_mode):
        return var
    else:
        pass

def read_sql(data, *args, **kwargs):
    """
    only read sql if env_mode is "dev", otherwise it is assumed that data is the json data passed as parameter
    """
    if is_dev_mode(env_mode):
        return pd.read_sql(*args, **kwargs)
    else:
        return pd.read_json(data)

In [ ]:
# GET DATA
sales_transactions = read_sql(data=sales_transactions, sql="SELECT * FROM etl.SalesTransactions st WHERE YEAR(st.PostingDate) > 2020", con=sql_connection_string)

In [ ]:
debug(sales_transactions)

In [ ]:
sales_transactions["PostingDate"] = pd.to_datetime(sales_transactions["PostingDate"])
sales_transactions["Year"] = sales_transactions["PostingDate"].dt.year
sales_transactions["Month"] = sales_transactions["PostingDate"].dt.month
grouped_sales = sales_transactions.groupby(by=["CompanyCode", "Year", "Month"])[["Quantity", "NetSalesEUR", "GrossProfitEUR"]].sum()

In [ ]:
debug(grouped_sales)

In [ ]:
# DO NOT DELETE THIS CELL
# ASSIGN YOUR RETURN VALUE HERE
return_value = grouped_sales.reset_index().to_json(orient="records")

In [ ]:
# DO NOT EDIT OR DELETE THIS CELL
return_value